## Downloading from Copernicus

In [1]:
%%capture
!pip install sentinelsat
!pip install pandas

In [2]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from getpass import getpass
import pandas

In [3]:
# Create an account on https://scihub.copernicus.eu/dhus/#/home and input your credentials here
username = input("Username: ")
password = getpass("Password: ")

# Image footprint generated from geojson.io
geojson_file = "./../../data/map.geojson"

# connect to the API
api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/apihub')

Username: 
Password: ········


In [4]:
starting_date = date(2020, 1, 1)
ending_date = date(2020, 1, 11)


# search by polygon, time, and Hub query keywords
footprint = geojson_to_wkt(read_geojson(geojson_file))
products = api.query(footprint,
                     date = (starting_date, ending_date),
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, 30))

UnauthorizedError: Invalid user name or password. Note that account creation and password changes may take up to a week to propagate to the 'https://apihub.copernicus.eu/apihub/' API URL you are using. Consider switching to 'https://scihub.copernicus.eu/dhus/' instead in the mean time.

In [ ]:
# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

# sort and limit to first sorted products
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
products_df_sorted = products_df_sorted.head(1)


In [ ]:
# download all results from the search
api.download_all(products_df_sorted.index)

# Downloading from Earth Explorer


2022 Montagnon Tristan (tristan.montagnon@univ-grenoble-alpes.fr || montagnon.tr@gmail.com)
Satellite images downloader

Questions, comments, and bug reports can be sent to:
[Montagnon Tristan](tristan.montagnon@univ-grenoble-alpes.fr)

## Installation and specificities
--------------
To have access to the data provided by Earth Explorer (EE):

- Create an account EE (https://ers.cr.usgs.gov/login)

- Request for Machine2Machine API access: https://ers.cr.usgs.gov/profile/access
    There is a small form to fill to complete this step. 
	Warning: This request will take a few days to be examined and accepted by EE staff

--------------
## To build the environment:
    
- Install landsatxplore package (with pip install landsatxplore): https://github.com/yannforget/landsatxplore 
- Adjustments to do in landsatxplore packages: 
    - In earthexplorer.py, comment / remove every line of "ncform":
    - Line 39, to be commented / removed
    - Line 43-44, to be commented / removed
    - Line 46, remove / comment the output ncform ONLY
    - Line 71, to be commented / removed

In the following example, we build a dictionary with the options required to download the dataset.

- As `dataset`, we chose Landsat-2, but you can change the dataset you want to download:
    
    - Landsat 5 TM Collection 1 Level 1 	landsat_tm_c1       AVAILABLE
    - Landsat 5 TM Collection 2 Level 1 	landsat_tm_c2_l1    AVAILABLE
    - Landsat 5 TM Collection 2 Level 2 	landsat_tm_c2_l2    AVAILABLE
    - Landsat 7 ETM+ Collection 1 Level 1 	landsat_etm_c1      AVAILABLE
    - Landsat 7 ETM+ Collection 2 Level 1 	landsat_etm_c2_l1   NOT AVAILABLE
    - Landsat 7 ETM+ Collection 2 Level 2 	landsat_etm_c2_l2   NOT AVAILABLE
    - Landsat 8 Collection 1 Level 1 	    landsat_8_c1        AVAILABLE
    - Landsat 8 Collection 2 Level 1 	    landsat_ot_c2_l1    NOT AVAILABLE
    - Landsat 8 Collection 2 Level 2 	    landsat_ot_c2_l2    NOT AVAILABLE
    - Sentinel 2A 	                        sentinel_2a         AVAILABLE

- By default, in the `location` field we set up to select a scene that contains the city of Grenoble.
- The `max_results_search` isn't supposed to be changed
- The `max_results_download` is the number of images you want to download
- `start_date` and `end_date` must be adapted to your will. Note that the dates must be in `YYYY-MM-DD` format
- With respect to `username` and `password`, your account linked to your username has to have acces to the M2M API. If you haven't done this step, please go this link: https://ers.cr.usgs.gov/profile/access

In [10]:
location = {
    'country': 'Grenoble', 
    'lat': 45.2, 
    'lon': 5.7
}

my_dict = {
    'location': location,
    'dataset': 'landsat_tm_c2_l2',
    'start_date': '2017-01-01',
    'end_date': '2022-01-01',
    'max_cloud_cover': 100.,
    'max_results_search': 100,
    'max_results_download': 2,
}

Username: TristanMTG


NameError: name 'getpass' is not defined

In [8]:
%%capture
# Installing the required package
!pip install landsatxplore

In [13]:
import time
import sys, os, pathlib
from getpass import getpass

from landsatxplore.api import API

sys.path.append(f"{pathlib.Path.cwd().parents[1]}")
from src.loading.earth_explorer_downloader import satellite_images_downloader_function, earth_explorer_patched

username = input("Username: ")
password = getpass("Password: ")

# The satellite images will be stored in './satellite_images/pathrow/
PATH_DATA = './'
PATH_SATELLITE_IMAGES = PATH_DATA + 'satellite_images/'
if os.path.exists(PATH_SATELLITE_IMAGES) is not True:
    os.mkdir(PATH_SATELLITE_IMAGES)
if os.path.exists(PATH_SATELLITE_IMAGES + location['country']) is not True:
    os.mkdir(PATH_SATELLITE_IMAGES + location['country'])

# Initialize a new API instance and get an access key
api = API(username, password)
ee = earth_explorer_patched(username, password)


start = time.time()
# This uses the satellite_images_downloader_function created in satellite_images_downloader_function.py
# The product you'll get is all the bands, metadata, etc. that are associated with your specified location
satellite_images_downloader_function(api=api, ee=ee, **my_dict)
end = time.time()
print(f"{(end - start):.3f} seconds")

# Closing the API
api.logout()
ee.logout()

Username: TristanMTG
Password: ········


Searching for scenes for the location: 
latitude: 45.2 
longitude: 5.7

0 scenes found.
No scene to download !
0.277 seconds
